<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE-11%5D%ED%8F%90%EB%A0%B4%EC%A7%84%EB%8B%A8%20%EC%84%B1%EB%8A%A5%EA%B0%9C%EC%84%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 루브릭

1.의료영상을 처리하는 CNN 기반 딥러닝 모델이 잘 구현되었다.
- 모델 학습이 안정적으로 수렴하는 것을 시각화를 통해 확인하였다.

2.데이터 준비, 모델구성 등의 과정의 다양한 실험이 체계적으로 수행되었다.
- regularization, augmentation 등의 기법의 사용 여부에 따른 모델 성능 측정이 ablation study 형태로 체계적으로 수행되었다.

3.실습코드를 잘 개선하여 폐렴 검출 정확도가 추가로 향상되었다.
- Accuracy 기준 85%에 도달하였다.